In [27]:
from pathlib import Path
import czifile
import tifffile
import napari
import pyclesperanto_prototype as cle
import numpy as np
import pandas as pd
from utils import check_filenames, segment_nuclei_2d, segment_marker_positive_nuclei

In [28]:
# From the results.csv, define the index of the image you want to explore in detail
image_index = 2

# Define the intensity threshold above which a cell is considered positive for a marker
neun_channel_threshold = 30
reelin_channel_threshold = 40
gad67_channel_threshold = 40

# Sets the amount of erosion that is applied to areas where the marker+ signal colocalizes with nuclear signal
# The higher the value, the stricter the conditions to consider a nuclei as marker+
neun_erosion_factor = 3
reelin_erosion_factor = 3
gad67_erosion_factor = 4

In [29]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/Reelin")
roi_directory_path = Path("./raw_data/Reelin/ROI")

# Define the subdirectories containing your data
subdirectories = ["Contra", "Ipsi", "Sham"]

# Create empty lists to store all image filepaths and ROIs within the dataset directory
images = []
rois = []

# Create an empty list to store all stats extracted from each image
stats = []

# Scan subdirectories and add paths to images fitting certain conditions
for subdir in subdirectories:
    # Construct the subdirectory path
    image_path = directory_path / subdir
    # Iterate through the .czi files in the subdirectories
    for file_path in image_path.glob("*.czi"):
        # Remove unwanted images
        if "AWT" not in str(file_path) and "BWT" not in str(file_path):
            images.append(str(file_path))

# Scan ROI directory and add paths to the list
for file_path in roi_directory_path.glob("*.tif"):
    # Remove unwanted images
        if "AWT" not in str(file_path) and "BWT" not in str(file_path):
            rois.append(str(file_path))

# Check if there is any missing ROI or image file in their corresponding directories
check_filenames(images, rois)

# Extract filenames without extensions and sort the lists so they appear in the same order
images_sorted = sorted(images, key=lambda x: Path(x).stem)
rois_sorted = sorted(rois, key=lambda x: Path(x).stem)

No files missing in images list.
No files missing in rois list.


In [30]:
# Select image to analyze based on the user-defined index
image_path = images_sorted[image_index]
roi_path = rois_sorted[image_index]

# Read path storing raw image and extract filename
file_path = Path(image_path)
filename = file_path.stem

# Get rid of double spaces in the filename
filename = filename.replace("  ", " ")

# Extract experimental conditions from the filename
descriptors = filename.split(" ")
condition = descriptors[0]
condition_nr = int(descriptors[1])
brain_location = descriptors[2]
mouse_id = int(descriptors[4])
slide = int(descriptors[5][-1])
tech_replica = int(descriptors[-1])

# Read image and ROI files into Numpy arrays
img = czifile.imread(image_path)
roi = tifffile.imread(roi_path)

# Remove singleton dimensions and perform MIP on input image
img = img.squeeze()
img_mip = np.max(img, axis=1)

# Perform MIP for the region of interest
roi_mip = np.max(roi, axis=0)

# We will create a mask where label_mip is greater than or equal to 1
mask = roi_mip >= 1

# Apply the mask to img_mip
masked_img = np.where(mask, img_mip, 0)

# Extract each of the channels separately
neun_mip = masked_img[0, :, :]
reelin_mip = masked_img[1, :, :]
gad67_mip = masked_img[2, :, :]
nuclei_mip = masked_img[3, :, :]

# Segment nuclei inside the ROI
nuclei_labels = segment_nuclei_2d(nuclei_mip)

# Dilate or erode nuclei to check for cytoplasmic or nuclear marker colocalization
cyto_nuclei_labels = cle.dilate_labels(nuclei_labels, radius=2)
cyto_nuclei_labels = cle.pull(cyto_nuclei_labels)
eroded_nuclei_labels = cle.erode_labels(nuclei_labels, radius=2)
eroded_nuclei_labels = cle.pull(eroded_nuclei_labels)

# Select marker positive nuclei
neun_tuple = segment_marker_positive_nuclei (nuclei_labels, neun_mip, neun_channel_threshold, neun_erosion_factor)
reelin_tuple = segment_marker_positive_nuclei (cyto_nuclei_labels, reelin_mip, reelin_channel_threshold, reelin_erosion_factor)
gad67_tuple = segment_marker_positive_nuclei (cyto_nuclei_labels, gad67_mip, gad67_channel_threshold, gad67_erosion_factor)

# Select Cajal cells by removing Reelin+ cells that are also positive for Neun

# Convert neun labels into a mask
neun_mask = neun_tuple[1] >= 1

# Check the shape of the arrays to ensure they match
assert reelin_tuple[1].shape == neun_mask.shape, "Label image and mask must have the same shape."

# Create a copy of the reelin label image to prevent modifying the original
cajal_cells = reelin_tuple[1].copy()

# Remove labels where the mask is True (or 1) by setting them to background values (0)
cajal_cells[neun_mask] = 0  

# Previous operation leaves residual cytoplasmic region of Neun+ cells (perform an erosion and dilation cycle)
cajal_cells = cle.erode_labels(cajal_cells, radius=2)
cajal_cells = cle.dilate_labels(cajal_cells, radius=2)
cajal_cells = cle.pull(cajal_cells)


In [31]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_mip)
viewer.add_image(roi_mip)
viewer.add_image(masked_img)
viewer.add_labels(nuclei_labels)
viewer.add_labels(cyto_nuclei_labels)
viewer.add_labels(eroded_nuclei_labels)
viewer.add_labels(neun_tuple[1], name="neun_+_nuclei")
viewer.add_labels(reelin_tuple[1], name="reelin_+_nuclei")
viewer.add_labels(gad67_tuple[1], name="gad67_+_nuclei")
viewer.add_labels(cajal_cells, name="cajal_cells")

<Labels layer 'cajal_cells' at 0x2305bdb3ee0>

In [32]:
# Count the number of positive cells for each marker (or cell population)
total_nuclei_count = len(np.unique(nuclei_labels)) - 1
neun_nuclei_count = len(np.unique(neun_tuple[1])) - 1
reelin_nuclei_count = len(np.unique(reelin_tuple[1])) - 1
gad67_nuclei_count = len(np.unique(gad67_tuple[1])) - 1
cajal_nuclei_count = len(np.unique(cajal_cells)) - 1

# Create a dictionary containing all extracted info per masked image
stats_dict = {
            "filename": filename,
            "condition": condition,
            "condition_nr": condition_nr,
            "brain_location": brain_location,
            "mouse_id": mouse_id,
            "slide_nr": slide,
            "tech_replica": tech_replica,
            "total_nuclei": total_nuclei_count,
            "neun+_nuclei": neun_nuclei_count,
            "reelin+_nuclei": reelin_nuclei_count,
            "gad67+_nuclei": gad67_nuclei_count,
            "cajal_nuclei": cajal_nuclei_count,
            "%_neun+_cells": (neun_nuclei_count * 100) / total_nuclei_count,
            "%_reelin+_cells": (reelin_nuclei_count * 100) / total_nuclei_count,
            "%_gad67+_cells": (gad67_nuclei_count * 100) / total_nuclei_count,
            "%_cajal_cells": (cajal_nuclei_count * 100) / total_nuclei_count
            }

# Append the current data point to the stats_list
stats.append(stats_dict)

stats_dict

{'filename': 'HI 1 Ipsilateral Mouse 8 Slide16 GAD67green NeuNpink Reelinred 40x 4x4 technical replica 1',
 'condition': 'HI',
 'condition_nr': 1,
 'brain_location': 'Ipsilateral',
 'mouse_id': 8,
 'slide_nr': 6,
 'tech_replica': 1,
 'total_nuclei': 3591,
 'neun+_nuclei': 1796,
 'reelin+_nuclei': 109,
 'gad67+_nuclei': 115,
 'cajal_nuclei': 68,
 '%_neun+_cells': 50.01392369813423,
 '%_reelin+_cells': 3.03536619326093,
 '%_gad67+_cells': 3.2024505708716235,
 '%_cajal_cells': 1.893622946254525}